In [1]:
import pandas as pd
import numpy as np 
import zipfile

In [2]:
#dataprocess/data/On_Time_Performance_1987_present_2003_1.zip
#On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2003_1.csv

#all_df = pd.DataFrame()
all_df = []
zip_prefix = "data/On_Time_Performance_1987_present_"
file_prefix = "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_"
for year in range(2017, 2019):
    for month in range(1, 13):
        full_zip_name = zip_prefix + str(year)+ "_" + str(month)+ ".zip"
        full_file_name = file_prefix + str(year)+ "_" + str(month)+ ".csv"
        with zipfile.ZipFile(full_zip_name) as z:
            with z.open(full_file_name) as f:
                df = pd.read_csv(f)
                final_df = df[["Year","Month","DayofMonth","DayOfWeek",
                               "FlightDate","Reporting_Airline","Origin",
                               "Dest","DepDelayMinutes","ArrDelayMinutes","Cancelled",
                               "CarrierDelay","WeatherDelay",
                               "NASDelay","SecurityDelay","LateAircraftDelay"]]
                df_after_drop_na = final_df.dropna(subset=['DepDelayMinutes', 'ArrDelayMinutes'])
                all_df.append(df_after_drop_na)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (48,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (48,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/an

In [3]:
#concatdinate all dataframe into big one  
result = pd.concat(all_df)
result_after_drop_na = result.dropna()

In [91]:
#result_after_drop_na.to_csv("clean_data/flight_data.csv")

In [4]:
#load airport coordinates file
coord_dr = pd.read_csv("clean_data/final_airport_data_info.csv")

In [5]:
#coord_dr['type'].value_counts()

In [6]:
# filter retain only small. medium and large airport
sml_df = coord_dr[coord_dr['type'].isin(["small_airport","medium_airport","large_airport"])]

In [7]:
# get list unique airport name
unique_airport_list = sml_df.drop_duplicates('iata_code')
len(unique_airport_list)

1866

In [8]:
#create a dictionary to store airport coordinates 
airport_coords = unique_airport_list[['iata_code','latitude_deg','longitude_deg']]
airport_coords.head(3)

,iata_code,latitude_deg,longitude_deg
0,LAX,33.942501,-118.407997
1,ORD,41.978600,-87.904800
2,JFK,40.639801,-73.778900


In [9]:
#convert to dictionary key
key = airport_coords.iata_code
lat =airport_coords.latitude_deg
long = airport_coords.longitude_deg
coordinate_dict = dict(zip(key,zip(lat,long)))
len(coordinate_dict)

1866

In [10]:
#define function to get latitude and longtitude 

def ori_lat(name,lookup_dict):
    try:
        return lookup_dict[name][0]
    except:
        pass

def ori_long(name,lookup_dict):
    try:
        return lookup_dict[name][1]
    except:
        pass

In [11]:
#create data series for origin and destination 
orgin_lat = result_after_drop_na['Origin']
orgin_long = result_after_drop_na['Origin']

des_lat = result_after_drop_na['Dest']
des_long = result_after_drop_na['Dest']

In [12]:
#Look up lat and long 
orgin_lat = orgin_lat.apply(lambda x: ori_lat(x,coordinate_dict))
orgin_long = orgin_long.apply(lambda x: ori_long(x,coordinate_dict))

des_lat = des_lat.apply(lambda x: ori_lat(x,coordinate_dict))
des_long = des_long.apply(lambda x: ori_long(x,coordinate_dict))

In [15]:
#append to final data 
result_after_drop_na['orgin_lat'] = orgin_lat
result_after_drop_na['orgin_long'] = orgin_long
result_after_drop_na['des_lat'] = des_lat
result_after_drop_na['des_long'] = des_long

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [17]:
result_after_drop_na.head(5)

,Year,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Origin,Dest,DepDelayMinutes,ArrDelayMinutes,Cancelled,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,orgin_lat,orgin_long,des_lat,des_long
0,2017,1,1,7,2017-01-01,AA,JFK,LAX,31.0,27.0,0.0,27.0,0.0,0.0,0.0,0.0,40.639801,-73.7789,33.942501,-118.407997
7,2017,1,8,7,2017-01-08,AA,JFK,LAX,4.0,21.0,0.0,0.0,4.0,17.0,0.0,0.0,40.639801,-73.7789,33.942501,-118.407997
8,2017,1,9,1,2017-01-09,AA,JFK,LAX,169.0,158.0,0.0,158.0,0.0,0.0,0.0,0.0,40.639801,-73.7789,33.942501,-118.407997
10,2017,1,11,3,2017-01-11,AA,JFK,LAX,0.0,18.0,0.0,0.0,0.0,18.0,0.0,0.0,40.639801,-73.7789,33.942501,-118.407997
13,2017,1,15,7,2017-01-15,AA,JFK,LAX,0.0,23.0,0.0,0.0,0.0,23.0,0.0,0.0,40.639801,-73.7789,33.942501,-118.407997


In [18]:
# Drop airport without coordinates and save to csv
result_after_drop_na = result_after_drop_na.dropna()
result_after_drop_na.to_csv("clean_data/final_flight_data.csv",index=False)